# Analysis of used car ads on eBay Kleinanzeigen

In this project, I will clean and analyze a dataset containing metrics on used car ads posted to eBay Kleinanzeigen, a classifieds section of the German eBay website. The dataset was originally posted to Kaggle but can now be found [here](https://data.world/data-society/used-cars-data). For this project, I'm using a version of the dataset that contains only 50,000 data points and which is dirtied a bit to get some practice cleaning data.

After cleaning the data, I tackle the following questions:

- What is the price distribution of the most common used car brands?
- What are the most common brand/model combinations?
- How much cheaper are used cars with versus without damage?

## Setup

In [89]:
import numpy as np
import pandas as pd

## Load data

In [90]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [91]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

Loading the data, we can see that we have 50000 rows and 20 columns, with columns including the seller for the ad, offer type, price, vehicle metrics, number of pictures, and more. We can see that we have a handful of columns with some null values: `vehicleType`, `gearbox`, `model`, `fuelType`, and `notRepairedDamage`.

## Rename columns

First, I'll rename some columns so that all columns are in snakecase:

In [92]:
autos.rename({'dateCrawled':'date_crawled',
              'offerType':'offer_type',
              'vehicleType':'vehicle_type',
              'yearOfRegistration':'registration_year',
              'powerPS':'power_PS',
              'monthOfRegistration':'registration_month',
              'fuelType':'fuel_type',
              'notRepairedDamage':'unrepaired_damage',
              'dateCreated':'ad_created',
              'nrOfPictures':'nr_of_pictures',
              'postalCode':'postal_code',
              'lastSeen':'last_seen'},
            axis=1, inplace=True)

print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


## Data cleaning

First, I inspect all the columns in the dataset to see if there are any numeric columns stored as text:

In [93]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


It looks like the `price` and `odometer` columns have numeric values stored as text, so I will convert and clean those values:

In [94]:
# convert price column to integers
autos['price'] = autos['price'].str.replace('$', '')
autos['price'] = autos['price'].str.replace(',', '')
autos['price'] = autos['price'].astype(int)
autos['price'].value_counts()

0        1421
500       781
1500      734
2500      643
1000      639
         ... 
20790       1
8970        1
846         1
2895        1
33980       1
Name: price, Length: 2357, dtype: int64

In [95]:
# convert odometer column and rename to `odometer_km`
autos['odometer'] = autos['odometer'].str.replace('km', '')
autos['odometer'] = autos['odometer'].str.replace(',', '')
autos['odometer'] = autos['odometer'].astype(int)
autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)
autos['odometer_km'].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

Next, I look at the `top` and `freq` entries to see if there are columns where all or almost all values are the same, as they might not be useful for analysis. It looks like the `seller` and `offer_type` columns have nearly all identical values, so we remove those:

In [96]:
autos.drop(['seller', 'offer_type'], axis=1, inplace=True)

### Removing outliers in the price column

I noticed when cleaning the `price` columns above that there were some unlikely values that might not be correct:

In [97]:
# view min/max/median etc
print(autos['price'].describe())

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


In [98]:
# view very high prices
print(autos['price'].value_counts().sort_index(ascending=False).head(20))

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64


In [99]:
# view very low prices
print(autos['price'].value_counts().sort_index(ascending=True).head(20))

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64


We are going to assume that prices below 200 and over 350000 are mistakes, so we'll remove the corresponding rows in the data:

In [100]:
autos = autos[autos['price'].between(200, 350000)]
autos['price'].describe()

count     47645.000000
mean       6000.707273
std        9110.783444
min         200.000000
25%        1300.000000
50%        3190.000000
75%        7500.000000
max      350000.000000
Name: price, dtype: float64

### Removing outliers in the odometer column

I also noticed some potentially erroneous values in teh `odometer_km` column. Below, I check for any potential outliers: 

In [101]:
autos['odometer_km'].describe()

count     47645.000000
mean     125887.501312
std       39482.911790
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [102]:
# view very high odometer values
print(autos['odometer_km'].value_counts().sort_index(ascending=False).head(10))

150000    30781
125000     4998
100000     2083
90000      1720
80000      1407
70000      1209
60000      1145
50000      1007
40000       813
30000       773
Name: odometer_km, dtype: int64


In [103]:
# view very low odometer values
print(autos['odometer_km'].value_counts().sort_index(ascending=True).head(10))

5000      720
10000     242
20000     747
30000     773
40000     813
50000    1007
60000    1145
70000    1209
80000    1407
90000    1720
Name: odometer_km, dtype: int64


I don't see any apparent outliers for this column, so we leave all entries for now.

### Cleaning date columns

There are three columns where dates are currently formatted as strings: `date_crawled`, `last_seen`, `ad_created`. I am going to extract the date from each column to explore the distributions of dates:

In [104]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025354
2016-03-06    0.014062
2016-03-07    0.035995
2016-03-08    0.033120
2016-03-09    0.033036
2016-03-10    0.032322
2016-03-11    0.032700
2016-03-12    0.036877
2016-03-13    0.015699
2016-03-14    0.036562
2016-03-15    0.034232
2016-03-16    0.029447
2016-03-17    0.031546
2016-03-18    0.012824
2016-03-19    0.034610
2016-03-20    0.037800
2016-03-21    0.037360
2016-03-22    0.032700
2016-03-23    0.032385
2016-03-24    0.029321
2016-03-25    0.031420
2016-03-26    0.032217
2016-03-27    0.031210
2016-03-28    0.035030
2016-03-29    0.033980
2016-03-30    0.033897
2016-03-31    0.031840
2016-04-01    0.033813
2016-04-02    0.035681
2016-04-03    0.038724
2016-04-04    0.036562
2016-04-05    0.013139
2016-04-06    0.003169
2016-04-07    0.001364
Name: date_crawled, dtype: float64

In [105]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038976
2016-04-04    0.036919
2016-04-05    0.011859
2016-04-06    0.003253
2016-04-07    0.001217
Name: ad_created, Length: 76, dtype: float64

In [106]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001091
2016-03-06    0.004303
2016-03-07    0.005373
2016-03-08    0.007178
2016-03-09    0.009613
2016-03-10    0.010473
2016-03-11    0.012299
2016-03-12    0.023906
2016-03-13    0.008899
2016-03-14    0.012530
2016-03-15    0.015720
2016-03-16    0.016287
2016-03-17    0.028083
2016-03-18    0.007283
2016-03-19    0.015615
2016-03-20    0.020653
2016-03-21    0.020506
2016-03-22    0.021408
2016-03-23    0.018491
2016-03-24    0.019624
2016-03-25    0.019079
2016-03-26    0.016686
2016-03-27    0.015511
2016-03-28    0.020758
2016-03-29    0.022185
2016-03-30    0.024578
2016-03-31    0.023864
2016-04-01    0.022899
2016-04-02    0.024808
2016-04-03    0.025102
2016-04-04    0.024620
2016-04-05    0.125470
2016-04-06    0.222437
2016-04-07    0.132669
Name: last_seen, dtype: float64

From the above, we can see that the ads included in the dataset were created in 2015 and 2016.

I'll also explore the `registration_year` column to explore the years of registration for the used cars:

In [107]:
autos['registration_year'].describe()

count    47645.000000
mean      2004.800084
std         88.423872
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Noticing that there is at least one outlier (the max value, 9999), I'll look for outliers and remove values accordingly:

In [108]:
# view very low registration years
autos['registration_year'].value_counts().sort_index(ascending=True).head(10)

1000    1
1001    1
1111    1
1800    2
1910    2
1927    1
1929    1
1931    1
1934    2
1937    4
Name: registration_year, dtype: int64

In [109]:
# view very high registration years
autos['registration_year'].value_counts().sort_index(ascending=False).head(15)

9999       3
9000       1
8888       1
6200       1
5911       1
5000       3
4800       1
4500       1
4100       1
2800       1
2019       1
2018     468
2017    1371
2016    1161
2015     376
Name: registration_year, dtype: int64

I'll treat years after 2016 and before 1910 as outliers or errors, since the ads were all created in 2015 or 2016, and it seems unlikely that cars built before 1910 would be for sale on eBay. I remove the corresponding rows from the dataset:

In [110]:
autos = autos[autos['registration_year'].between(1910, 2016)]
autos['registration_year'].describe()

count    45786.000000
mean      2002.993098
std          7.113188
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

## What is the price distribution of the most common used car brands?

Next, I'll explore the prices of used cars according to brand. I'll start by exploring what brands of cars are in the dataset:

In [111]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.211244
bmw               0.111257
opel              0.105906
mercedes_benz     0.097628
audi              0.087603
ford              0.068733
renault           0.046543
peugeot           0.029944
fiat              0.025095
seat              0.018171
skoda             0.016512
nissan            0.015289
mazda             0.015136
smart             0.014371
citroen           0.014131
toyota            0.012930
hyundai           0.010112
sonstige_autos    0.009544
volvo             0.009195
mini              0.008889
mitsubishi        0.008168
honda             0.007928
kia               0.007142
alfa_romeo        0.006683
porsche           0.006094
suzuki            0.005810
chevrolet         0.005722
chrysler          0.003560
dacia             0.002686
daihatsu          0.002490
jeep              0.002315
land_rover        0.002140
subaru            0.002097
saab              0.001660
jaguar            0.001529
daewoo            0.001485
rover             0.001332
t

For simplicity, I'm going to examine prices for used cars from only brands present for at least 5% of ads:

In [112]:
brands_sorted = autos['brand'].value_counts(normalize=True)
brands = brands_sorted[brands_sorted >= .05].index.values
print(brands)

['volkswagen' 'bmw' 'opel' 'mercedes_benz' 'audi' 'ford']


Next, I loop over this subset of 6 brands and find the mean used car price for each brand:

In [113]:
prices_by_brand = {}
for b in brands:
    prices_by_brand[b] = autos.loc[autos['brand'] == b, 'price'].mean()
    
print(prices_by_brand)

{'volkswagen': 5506.4409636062865, 'bmw': 8402.666077738515, 'opel': 3077.5772324190557, 'mercedes_benz': 8691.71744966443, 'audi': 9406.093243580155, 'ford': 3883.292024149984}


The result indicates that Audi, Mercedes Benz, and BMW cars were the most expensive, whereas Ford and Opel cars were the least expensive. Volkswagen fell between these two groupings.

## Is there a link between price and mileage for the top used car brands?

Next, I'd like to see if there is a link between price and mileage, and whether that varies by brand. I will calculate the mean mileage and price for each of the 6 brands identified above:

In [114]:
mileage_by_brand = {}
# calculate mean mileage for each of the top brands
for b in brands:
    mileage_by_brand[b] = autos.loc[autos['brand'] == b, 'odometer_km'].mean()

In [115]:
# join mean mileage and mean price data in a single dataframe
price_series = pd.Series(prices_by_brand)
mileage_series = pd.Series(mileage_by_brand)

data_brands = pd.DataFrame(price_series, columns = ['mean_price'])
data_brands = data_brands.assign(mean_mileage = mileage_series)
print(data_brands)

                mean_price   mean_mileage
volkswagen     5506.440964  128774.813896
bmw            8402.666078  132792.500982
opel           3077.577232  129231.800371
mercedes_benz  8691.717450  131091.722595
audi           9406.093244  129260.782847
ford           3883.292024  124095.964411


The mean mileage is very similar across brands. So, it doesn't seem like there is a clear link between mileage and mean price. Brand seems to be more of a driver of price.

## What are the most common brand/model combinations?

Next, I want to figure out what the most common brand/model combinations in the dataset. I'm only examining combinations for the top 6 brands identified above:

In [116]:
models_by_brand = {}
for b in brands:
    # choose the top 3 most common models
    top_models = np.array(autos.loc[autos['brand'] == b, 'model'].value_counts(normalize=True).head(3).index.values)
    print(b + ':')
    print(top_models)
    print('')

volkswagen:
['golf' 'polo' 'passat']

bmw:
['3er' '5er' '1er']

opel:
['corsa' 'astra' 'vectra']

mercedes_benz:
['c_klasse' 'e_klasse' 'a_klasse']

audi:
['a4' 'a3' 'a6']

ford:
['focus' 'fiesta' 'mondeo']



## How much cheaper are cars with damage vs. without damage?

Finally, I compare the distribution of prices for cars with and without damage:

In [117]:
autos['unrepaired_damage'].unique()

array(['nein', nan, 'ja'], dtype=object)

In [118]:
price_by_repairstatus = {}
for value in ['ja', 'nein']:
    price_by_repairstatus[value] = np.array([
        autos.loc[autos['unrepaired_damage'] == value, 'price'].mean(),
        autos.loc[autos['unrepaired_damage'] == value, 'price'].median(),
        autos.loc[autos['unrepaired_damage'] == value, 'price'].min(),
        autos.loc[autos['unrepaired_damage'] == value, 'price'].max()
    ]
    )
    print('Unrepaired damage -- ' + value + ':')
    print(price_by_repairstatus[value].astype(int))

Unrepaired damage -- ja:
[ 2361  1100   200 44200]
Unrepaired damage -- nein:
[  7198   4200    200 350000]


Not surprisingly, the mean, median and maximum price for cars without unrepaired damage is higher than that for cars with repaired damage. The minimum price for cars with and without damage is the same.